In [9]:
import pandas as pd

#saved_x_tuesday.csv

df = pd.read_csv('flatten_data/saved_x_tuesday.csv',index_col=None)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.head(5)


,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.929828,-0.656845,0.659908,0.856356,-0.360853,0.196556,0.902478,-0.664118,0.189856,1.005288,...,1.015059,-1.675480,-1.945418,-0.317206,0.068530,-1.420136,0.970085,-1.129397,-0.661374,-1.041526
1,-0.056936,-0.360656,-0.780846,0.625369,0.308352,-1.207259,-0.028001,0.081051,-0.310865,0.426681,...,-1.340657,-1.282031,2.199889,1.313291,-0.594478,-1.589383,-0.237698,-0.849886,-0.121543,0.623533
2,0.490560,0.762371,-0.920771,0.216243,1.708066,-0.443268,0.518200,0.525918,0.370657,-0.779526,...,0.086032,-0.402748,0.652358,0.500372,0.695925,-1.376089,0.261011,0.044002,-0.479850,0.067027
3,0.994773,-0.568013,0.229133,-0.181303,-0.289275,0.438698,1.351305,-0.413223,-0.071022,-0.090384,...,-0.000397,-0.282919,-0.971175,0.087314,0.618838,1.372782,1.576300,0.011231,0.701612,1.439610
4,0.833888,-0.557214,0.995512,1.081478,-0.670258,0.533618,0.856240,-0.467297,0.942266,0.911166,...,-0.972883,-2.482897,-0.776684,0.761242,0.243920,0.604705,-0.222797,-2.829603,-0.909716,0.553585


In [14]:
y = pd.read_csv('flatten_data/saved_y_tuesday.csv',index_col=None)
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y.head(5)

,action_label
0,0
1,0
2,0
3,0
4,0


In [20]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np

INPUT_SIZE = 240
OUTPUT_SIZE = 9

class Model:
    def __init__(self, bitfile, paramfile):
        # Initialize Overlay & DMA
        self.overlay = Overlay(bitfile)
        self.dma = self.overlay.axi_dma_0
        
        # Load weights and bias
        f = open(paramfile, "r")
        self.params = f.read().split(',')
        for i in range(len(self.params)):
            self.params[i] = np.float32(self.params[i])
        self.numofparams = len(self.params)
        print(self.numofparams)
        
        # Initialize DMA buffer
        print("Total input length: ",self.numofparams+INPUT_SIZE)
        self.input_buffer = allocate(shape=(self.numofparams+INPUT_SIZE,), dtype=np.float32)
        for i in range (self.numofparams):
            self.input_buffer[i] = self.params[i]
        self.res = allocate(shape=(2*OUTPUT_SIZE,), dtype=np.float32)
        
    def classify(self, in_x):
        for i in range(INPUT_SIZE):
            self.input_buffer[i+self.numofparams] = in_x[i]

        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.recvchannel.transfer(self.res)
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        max_index = np.argmax(self.res)
        return max_index, self.res[max_index]
        

In [21]:
import time

mlp = Model("bitstreams/mlpv4_4.bit", "mlpv4_4_stdscaler.csv")

correct = 0
total = 0
start_time = time.time()
for i in range(df.values.shape[0]):
    input_x = df.values[i]
    pred, prob = mlp.classify(input_x)
    print(pred,prob, y.values[i])
    if (pred == y.values[i]):
        correct += 1
    total += 1
print("-----%s ms in average elapsed for each transfer-----" %(1000*(time.time()-start_time)/total))
print("TOTAL ACCURACY: ", correct/float(total))


39689
Total input length:  39929
0 0.9954928 [0]
0 0.99779314 [0]
0 0.9936087 [0]
0 0.98297536 [0]
0 0.9773783 [0]
0 0.9833889 [0]
0 0.91961676 [0]
0 0.4791058 [0]
0 0.994648 [0]
4 0.9547063 [0]
0 0.76253843 [0]
0 0.94991165 [0]
0 0.9915864 [0]
0 0.9987046 [0]
0 0.9991338 [0]
0 0.9966451 [0]
0 0.99839044 [0]
0 0.99945813 [0]
0 0.99788594 [0]
0 0.97986716 [0]
0 0.99649745 [0]
0 0.9836825 [0]
0 0.99284047 [0]
0 0.9998071 [0]
0 0.9957246 [0]
0 0.998113 [0]
0 0.999695 [0]
0 0.99878204 [0]
0 0.99847263 [0]
0 0.99580073 [0]
0 0.9972889 [0]
0 0.9848542 [0]
0 0.99647075 [0]
0 0.9838151 [0]
0 0.9907562 [0]
0 0.9921734 [0]
0 0.9911786 [0]
0 0.997443 [0]
0 0.9973077 [0]
0 0.99867344 [0]
0 0.9961051 [0]
0 0.86923087 [0]
0 0.99516153 [0]
0 0.9996634 [0]
0 0.96172374 [0]
0 0.9998176 [0]
0 0.9798889 [0]
0 0.9983983 [0]
0 0.99994713 [0]
0 0.9939104 [0]
0 0.9992339 [0]
0 0.99910736 [0]
0 0.97596294 [0]
0 0.9965577 [0]
0 0.9914306 [0]
0 0.8317448 [0]
0 0.999716 [0]
0 0.99521697 [0]
0 0.9719101 [0]
0 0.9

0 0.9975921 [0]
0 0.99801946 [0]
0 0.99203587 [0]
0 0.9990115 [0]
0 0.9992805 [0]
0 0.9987745 [0]
0 0.9992044 [0]
0 0.99742544 [0]
0 0.9934316 [0]
0 0.9994288 [0]
0 0.9978738 [0]
0 0.99929386 [0]
0 0.98984706 [0]
0 0.99422634 [0]
0 0.99538034 [0]
0 0.9967341 [0]
0 0.9974386 [0]
0 0.9996417 [0]
0 0.9925048 [0]
0 0.997164 [0]
0 0.9998949 [0]
0 0.99441177 [0]
0 0.99865913 [0]
0 0.99799997 [0]
0 0.9995832 [0]
0 0.99402547 [0]
0 0.9972371 [0]
0 0.99997723 [0]
0 0.99967444 [0]
0 0.99987394 [0]
0 0.9999043 [0]
0 0.9945742 [0]
0 0.99814814 [0]
0 0.9987903 [0]
0 0.99621457 [0]
0 0.9979817 [0]
0 0.9999149 [0]
0 0.9967425 [0]
0 0.9998452 [0]
0 0.9916614 [0]
0 0.9998427 [0]
0 0.99898887 [0]
0 0.9978936 [0]
0 0.9999048 [0]
0 0.99936986 [0]
0 0.99967587 [0]
0 0.99955726 [0]
0 0.99869204 [0]
0 0.9999076 [0]
0 0.999656 [0]
0 0.9991383 [0]
0 0.99969316 [0]
0 0.9996225 [0]
0 0.99982876 [0]
0 0.9971048 [0]
0 0.9954397 [0]
0 0.99957323 [0]
0 0.9997761 [0]
0 0.99928576 [0]
0 0.9998592 [0]
0 0.9982196 [0]
0

1 0.989403 [1]
1 0.98407495 [1]
1 0.9991961 [1]
1 0.9767028 [1]
1 0.98949546 [1]
1 0.99973214 [1]
1 0.99879414 [1]
1 0.9972263 [1]
1 0.95744133 [1]
1 0.9873216 [1]
1 0.94303286 [1]
1 0.9944864 [1]
1 0.9983256 [1]
1 0.99964833 [1]
1 0.97603714 [1]
1 0.9997764 [1]
1 0.99920154 [1]
1 0.93347496 [1]
1 0.98216426 [1]
1 0.9995782 [1]
1 0.9996151 [1]
1 0.9999279 [1]
1 0.99697894 [1]
1 0.9967989 [1]
1 0.9968177 [1]
1 0.8603638 [1]
1 0.9868497 [1]
1 0.996659 [1]
1 0.9999593 [1]
1 0.9835441 [1]
1 0.99835944 [1]
1 0.99885887 [1]
1 0.998151 [1]
1 0.8884415 [1]
1 0.999841 [1]
1 0.9956802 [1]
1 0.9909172 [1]
1 0.99162346 [1]
1 0.9997912 [1]
1 0.9942303 [1]
1 0.9976542 [1]
1 0.9952946 [1]
1 0.9499538 [1]
1 0.9424578 [1]
1 0.9980916 [1]
1 0.99818766 [1]
1 0.99852943 [1]
1 0.9949388 [1]
1 0.9996448 [1]
1 0.999215 [1]
1 0.9379857 [1]
1 0.99844134 [1]
1 0.999503 [1]
1 0.9930635 [1]
1 0.99240494 [1]
1 0.9376769 [1]
1 0.9875453 [1]
1 0.9935444 [1]
1 0.9887018 [1]
1 0.99224234 [1]
1 0.9888023 [1]
1 0.999627

2 0.99255645 [2]
2 0.9570497 [2]
2 0.9912631 [2]
2 0.9012089 [2]
2 0.9561622 [2]
2 0.99985176 [2]
2 0.9198854 [2]
2 0.9999512 [2]
2 0.99364394 [2]
2 0.9828662 [2]
2 0.9847572 [2]
2 0.98808044 [2]
2 0.92128086 [2]
2 0.9985 [2]
2 0.9950339 [2]
2 0.98722476 [2]
2 0.93821347 [2]
2 0.9987301 [2]
2 0.93197113 [2]
2 0.9531843 [2]
2 0.9984362 [2]
2 0.99991226 [2]
2 0.99777526 [2]
2 0.99636286 [2]
2 0.9665671 [2]
2 0.9987728 [2]
2 0.9959896 [2]
2 0.99908346 [2]
2 0.8876794 [2]
2 0.9982397 [2]
2 0.9999488 [2]
2 0.9156823 [2]
2 0.9987889 [2]
2 0.9988484 [2]
2 0.99748087 [2]
2 0.9859293 [2]
2 0.98813474 [2]
2 0.99084026 [2]
2 0.9989929 [2]
2 0.9866315 [2]
2 0.99815357 [2]
2 0.9972272 [2]
2 0.9998749 [2]
2 0.7864062 [2]
2 0.8821873 [2]
2 0.9898571 [2]
2 0.9219849 [2]
2 0.8138976 [2]
2 0.9874293 [2]
2 0.9951782 [2]
2 0.96646136 [2]
2 0.99253875 [2]
2 0.9988942 [2]
2 0.99858826 [2]
2 0.99843794 [2]
2 0.9947853 [2]
2 0.98122364 [2]
2 0.9900933 [2]
2 0.99948895 [2]
2 0.97907174 [2]
2 0.9989157 [2]
2 0.

2 0.9963635 [2]
4 0.9883175 [2]
2 0.7209186 [2]
2 0.94953865 [2]
2 0.73243684 [2]
2 0.9974239 [2]
2 0.96781415 [2]
2 0.5676044 [2]
2 0.993685 [2]
2 0.9631483 [2]
2 0.9889896 [2]
2 0.9870823 [2]
2 0.9797758 [2]
2 0.98021835 [2]
2 0.9637194 [2]
2 0.808592 [2]
2 0.73990506 [2]
2 0.94176835 [2]
2 0.9257453 [2]
2 0.98865163 [2]
2 0.9994606 [2]
2 0.56204915 [2]
2 0.9228357 [2]
2 0.9744433 [2]
2 0.7065829 [2]
7 0.46991283 [2]
2 0.95232695 [2]
2 0.9844906 [2]
2 0.9956971 [2]
2 0.983584 [2]
2 0.9405669 [2]
2 0.990691 [2]
2 0.9691851 [2]
2 0.992598 [2]
2 0.91870326 [2]
2 0.975969 [2]
2 0.9985157 [2]
4 0.61292064 [2]
2 0.9859836 [2]
2 0.708913 [2]
2 0.82211006 [2]
2 0.9903878 [2]
2 0.99355024 [2]
4 0.8125449 [2]
2 0.95949125 [2]
2 0.9981626 [2]
2 0.9793239 [2]
2 0.9987706 [2]
2 0.48797986 [2]
2 0.9997275 [2]
2 0.99845237 [2]
2 0.9985322 [2]
2 0.964339 [2]
2 0.50439364 [2]
2 0.98243546 [2]
2 0.9999063 [2]
2 0.99908733 [2]
4 0.7086322 [2]
2 0.99350286 [2]
2 0.9991731 [2]
2 0.8779812 [2]
2 0.9834691

...

5 0.9999653 [5]
5 0.9988417 [5]
5 0.9999346 [5]
5 0.99781686 [5]
5 0.9999937 [5]
5 0.9994172 [5]
5 0.99992186 [5]
5 0.9999811 [5]
5 0.9999945 [5]
5 0.99985325 [5]
5 0.99999934 [5]
5 0.99889976 [5]
5 0.9998438 [5]
5 0.99969614 [5]
5 0.9991187 [5]
5 0.9999751 [5]
5 0.9527249 [5]
5 0.99546564 [5]
5 0.99999285 [5]
5 0.999983 [5]
5 0.9974676 [5]
5 0.99997836 [5]
5 0.999009 [5]
5 0.9999939 [5]
5 0.99989766 [5]
5 0.9998651 [5]
5 0.9999369 [5]
5 0.9999879 [5]
5 0.99966514 [5]
5 0.9999668 [5]
5 0.9999955 [5]
5 0.9999898 [5]
5 0.99328274 [5]
5 0.99999785 [5]
5 0.99770254 [5]
5 0.99950325 [5]
5 0.9999865 [5]
5 0.99995726 [5]
5 0.9999962 [5]
5 0.9999961 [5]
5 0.99935865 [5]
5 0.99518687 [5]
5 0.9999976 [5]
5 0.99998266 [5]
5 0.99986523 [5]
5 0.99999 [5]
5 0.9999909 [5]
5 0.9999993 [5]
5 0.88594383 [5]
5 0.99999446 [5]
5 0.9856342 [5]
5 0.99741083 [5]
5 0.99988943 [5]
5 0.999991 [5]
5 0.99998426 [5]
5 0.99869597 [5]
5 0.99261755 [5]
5 0.9999937 [5]
5 0.9999084 [5]
5 0.9994954 [5]
5 0.99999785 [5]
5

6 0.99907196 [6]
6 0.99997526 [6]
6 0.99831086 [6]
6 0.9992226 [6]
6 0.9996384 [6]
6 0.99869746 [6]
6 0.99880534 [6]
6 0.9996856 [6]
6 0.9995586 [6]
6 0.999831 [6]
6 0.9992203 [6]
6 0.9998432 [6]
6 0.9757667 [6]
6 0.9999386 [6]
6 0.99965 [6]
6 0.99976355 [6]
6 0.99998623 [6]
6 0.9997426 [6]
6 0.9981869 [6]
6 0.9990224 [6]
6 0.9996781 [6]
6 0.9999773 [6]
6 0.9985155 [6]
6 0.9974457 [6]
6 0.9995148 [6]
6 0.9999324 [6]
6 0.99896795 [6]
6 0.9963552 [6]
6 0.9972191 [6]
6 0.9337383 [6]
6 0.9998709 [6]
6 0.9992119 [6]
6 0.9904703 [6]
6 0.9990635 [6]
6 0.99946594 [6]
6 0.99978834 [6]
6 0.99993664 [6]
6 0.9858682 [6]
6 0.9999672 [6]
6 0.9945222 [6]
6 0.9905328 [6]
6 0.99739575 [6]
6 0.9989707 [6]
6 0.9998232 [6]
6 0.9999717 [6]
6 0.9998536 [6]
6 0.9955867 [6]
6 0.99974144 [6]
6 0.98618305 [6]
6 0.9913122 [6]
6 0.99849856 [6]
6 0.9997349 [6]
6 0.9999578 [6]
6 0.9997765 [6]
6 0.9999803 [6]
6 0.9999507 [6]
6 0.999694 [6]
6 0.999865 [6]
6 0.9997609 [6]
6 0.9974414 [6]
6 0.99992186 [6]
6 0.9998272 [

7 0.998805 [7]
7 0.9995729 [7]
7 0.9979005 [7]
7 0.9981379 [7]
7 0.99968374 [7]
7 0.999957 [7]
7 0.99804455 [7]
7 0.9974538 [7]
7 0.9999635 [7]
7 0.999953 [7]
7 0.99985015 [7]
7 0.99945104 [7]
7 0.99996626 [7]
7 0.99997836 [7]
7 0.9995512 [7]
7 0.9999967 [7]
7 0.99979323 [7]
7 0.99998087 [7]
7 0.9996372 [7]
7 0.9999571 [7]
7 0.9998384 [7]
7 0.9998557 [7]
7 0.99997956 [7]
7 0.99989164 [7]
7 0.9978027 [7]
7 0.99989736 [7]
7 0.9999299 [7]
7 0.9999565 [7]
7 0.99859357 [7]
7 0.99995697 [7]
7 0.999837 [7]
7 0.9999306 [7]
7 0.9894928 [7]
7 0.9994035 [7]
7 0.99973756 [7]
7 0.9999951 [7]
7 0.99998564 [7]
7 0.999944 [7]
7 0.99999344 [7]
7 0.99964905 [7]
7 0.99252564 [7]
7 0.9803088 [7]
7 0.904872 [7]
7 0.9900537 [7]
7 0.9986369 [7]
7 0.999143 [7]
7 0.99938715 [7]
7 0.9951248 [7]
7 0.57249784 [7]
7 0.99386716 [7]
7 0.99998844 [7]
7 0.99782544 [7]
7 0.9998877 [7]
7 0.99842805 [7]
7 0.9999872 [7]
7 0.999813 [7]
7 0.9998686 [7]
7 0.99937785 [7]
7 0.99925923 [7]
7 0.9973903 [7]
7 0.98025936 [7]
7 0.9

7 0.99993616 [7]
7 0.99976635 [7]
7 0.99980754 [7]
7 0.99997103 [7]
7 0.99999493 [7]
7 0.99992895 [7]
7 0.99998677 [7]
7 0.99999756 [7]
7 0.9999397 [7]
7 0.9999323 [7]
7 0.99974304 [7]
7 0.9999786 [7]
7 0.99999344 [7]
7 0.9997823 [7]
7 0.9998538 [7]
7 0.9998304 [7]
7 0.99999845 [7]
7 0.99998456 [7]
7 0.99999946 [7]
7 0.99999434 [7]
7 0.9999743 [7]
7 0.9998893 [7]
7 0.9999733 [7]
7 0.99992615 [7]
7 0.9999956 [7]
7 0.99978864 [7]
7 0.99873596 [7]
7 0.99999934 [7]
7 0.9999932 [7]
7 0.99999434 [7]
7 0.999935 [7]
7 0.9999632 [7]
7 0.99725693 [7]
7 0.9990973 [7]
7 0.99751 [7]
7 0.99993247 [7]
7 0.99961066 [7]
7 0.9999919 [7]
7 0.9993557 [7]
7 0.9995145 [7]
7 0.99999416 [7]
7 0.9999902 [7]
7 0.9978215 [7]
7 0.99783164 [7]
7 0.99716556 [7]
7 0.9999796 [7]
7 0.9976886 [7]
7 0.998132 [7]
7 0.9997059 [7]
7 0.99999654 [7]
7 0.99978477 [7]
7 0.9982868 [7]
7 0.9999903 [7]
7 0.9998442 [7]
7 0.99998635 [7]
7 0.9922312 [7]
7 0.9999799 [7]
7 0.9997421 [7]
7 0.9993444 [7]
7 0.9998626 [7]
7 0.99748117 [7]

8 0.9960487 [8]
8 0.9834049 [8]
8 0.9982317 [8]
8 0.9947431 [8]
8 0.99798983 [8]
8 0.9988695 [8]
8 0.9992089 [8]
8 0.9919904 [8]
8 0.974229 [8]
8 0.99598867 [8]
8 0.99533474 [8]
8 0.99291366 [8]
8 0.99322265 [8]
8 0.99231803 [8]
8 0.98643565 [8]
8 0.9852668 [8]
8 0.9547521 [8]
8 0.91637814 [8]
8 0.98241585 [8]
8 0.9301998 [8]
8 0.99207836 [8]
8 0.7523543 [8]
8 0.8825118 [8]
8 0.9242812 [8]
8 0.98054683 [8]
8 0.9900521 [8]
8 0.94582564 [8]
8 0.8820215 [8]
8 0.92495114 [8]
8 0.98410565 [8]
8 0.9868502 [8]
8 0.9923471 [8]
8 0.9735877 [8]
8 0.99039197 [8]
8 0.9937337 [8]
8 0.996132 [8]
8 0.88261515 [8]
8 0.9925144 [8]
8 0.99014884 [8]
8 0.90454125 [8]
8 0.969381 [8]
8 0.9861937 [8]
8 0.95673305 [8]
8 0.9870725 [8]
8 0.9237551 [8]
8 0.99498767 [8]
8 0.96800417 [8]
8 0.99706864 [8]
8 0.99516904 [8]
8 0.99858224 [8]
8 0.9918326 [8]
8 0.98525107 [8]
8 0.9332509 [8]
8 0.95621246 [8]
8 0.9764208 [8]
8 0.631255 [8]
8 0.9891203 [8]
8 0.98771137 [8]
8 0.9935697 [8]
8 0.9807922 [8]
8 0.9884292 [8]
8